In [9]:
import contextlib
import io

import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 

from scipy.interpolate import interp1d

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Bidirectional, GRU

In [10]:
def split_dataframe(df, train_size=0.7, val_size=0.2):
    """
    Découpe un DataFrame temporel en trois parties : train, val, test,
    en respectant l'ordre chronologique.

    Args:
        df (pd.DataFrame): Données à découper (indexé ou non par le temps)
        train_size (float): Proportion pour l'ensemble d'entraînement
        val_size (float): Proportion pour la validation

    Returns:
        df_train, df_val, df_test (DataFrames)
    """
    n = len(df)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))

    df_train = df.iloc[:train_end]
    df_val = df.iloc[train_end:val_end]
    df_test = df.iloc[val_end:]

    print(f"Train size : {len(df_train)}")
    print(f"Val size : {len(df_val)}")
    print(f"Test size : {len(df_test)}")

    return df_train.copy(), df_val.copy(), df_test.copy()

In [11]:
def evaluate_model(y_true, y_pred):
    """
    Calcule et affiche les métriques MAE, RMSE et R² entre les vraies valeurs et les prédictions.

    Args:
        y_true (array-like): Valeurs réelles.
        y_pred (array-like): Valeurs prédites.
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R2 Score: {r2:.4f}')
    
    return mae, rmse, r2

In [12]:
def create_dataset(X, y, time_steps=1):  
    Xs, ys = [], []   
    for i in range(len(X) - time_steps):   
        v = X.iloc[i:(i + time_steps)].values 
        Xs.append(v)      
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

    """
This function prepares the input features and target values in the format required for training a recurrent neural network (RNN) or LSTM model for sequential prediction tasks. It creates sequences of input features and their corresponding target values, which can be fed into the model during training.

    - X: This parameter represents the input features, typically a pandas DataFrame containing multiple time-series variables such as temperature, humidity, etc.
    - y: This parameter represents the target values, which are typically the values we want to predict based on the input features.
    - time_steps: This parameter defines the length of each sequence. It determines how many data points from the past will be used to predict the next data point. For example, if time_steps is set to 3, the function will create sequences of three consecutive data points as input features and the next data point as the target value.
    """

In [13]:
def build_and_train_bilstm(X_train, y_train, X_val, y_val, sequence_length, units=100, 
                           activation='tanh', epochs=30, batch_size=32, patience=10):
    """
    Construit, entraîne et évalue un modèle BiLSTM avec early stopping.

    Args:
        X_train (ndarray): Données d'entraînement (3D).
        y_train (ndarray): Cibles d'entraînement.
        X_val (ndarray): Données de validation (3D).
        y_val (ndarray): Cibles de validation.
        sequence_length (int): Longueur des séquences en entrée.
        units (int): Nombre de neurones dans la couche LSTM.
        activation (str): Fonction d'activation de la couche LSTM.
        epochs (int): Nombre d’époques d’entraînement.
        batch_size (int): Taille de lot pour l’entraînement.
        patience (int): Patience pour l’early stopping.

    Returns:
        model: Le modèle entraîné.
        history: L'historique d'entraînement.
        val_loss: La perte de validation finale.
    """
    model = Sequential()
    model.add(Bidirectional(LSTM(units, activation=activation, input_shape=(sequence_length, X_train.shape[2]))))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )

    val_loss = model.evaluate(X_val, y_val, verbose=0)
    display(f'Validation Loss: {val_loss}')

    return model, history, val_loss


In [14]:
def load_data_drias(path_fichier_excel):
    """
    Charge un fichier Excel DRIAS avec une colonne 'Date' au format '%d/%m/%Y',
    et retourne un DataFrame avec la date en index.

    Args:
        path_fichier_excel (str): Chemin vers le fichier Excel.

    Returns:
        pd.DataFrame: Données DRIAS avec l'index daté.
    """
    df = pd.read_excel(path_fichier_excel)
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df.set_index('Date', inplace=True)
    display(df)
    
    return df

In [15]:
def predict_and_inverse_transform(model, X, y, target_transformer):
    """
    Effectue la prédiction avec le modèle donné et applique l'inverse de la transformation 
    sur les prédictions et les vraies valeurs cibles.

    Args:
        model: Modèle entraîné (ex. BiLSTM).
        X_val: Données d'entrée de validation.
        y_val: Vraies valeurs cibles de validation.
        target_transformer: Transformateur utilisé pour normaliser les cibles (ex. MinMaxScaler).

    Returns:
        Tuple (y_pred_original_scale, y_true_original_scale)
    """
    # Prédiction
    y_pred = model.predict(X)

    # Inversion de la transformation des prédictions
    y_pred_inv = target_transformer.inverse_transform(y_pred)

    # Reshape puis inversion de la transformation des vraies valeurs
    y = y.reshape(-1, 1)
    y_val_inv = target_transformer.inverse_transform(y)

    return y_pred_inv, y_val_inv

In [16]:
def convertir_q_en_rh(q_kgkg, temperature_C, pression_hPa=1013.25):
    """
    Convertit une série d'humidité spécifique (kg/kg) et de température (°C)
    en humidité relative (%) en supposant une pression constante.

    Paramètres :
        q_kgkg : pd.Series ou np.array d'humidité spécifique (kg/kg)
        temperature_C : pd.Series ou np.array de température (°C)
        pression_hPa : pression atmosphérique en hPa (par défaut = 1013.25)

    Retour :
        pd.Series ou np.array d'humidité relative (%) — même type que l'entrée
    """
    q = np.asarray(q_kgkg)
    T = np.asarray(temperature_C)

    # Pression partielle de vapeur d'eau (e) [hPa]
    e = (q * pression_hPa) / (0.622 + 0.378 * q)

    # Pression de vapeur saturante (e_s) [hPa] — formule de Tetens
    e_s = 6.112 * np.exp((17.67 * T) / (T + 243.5))

    # Humidité relative RH [%]
    RH = 100 * e / e_s
    RH = np.clip(RH, 0, 100)

    # Renvoyer dans le même format que l'entrée
    if isinstance(q_kgkg, pd.Series):
        return pd.Series(RH, index=q_kgkg.index, name='RH_%')
    else:
        return RH

##### **Dataset DRIAS RCP**

In [17]:
data_rcp_2_6 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp2.6_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
Date,,,,,,,,,,,,
2006-01-01,49.2427,4.03882,-3.15,4.59,-0.04,0.000,0.000,0.003508,53.716,265.013,2.181,0.155
2006-01-02,49.2427,4.03882,0.24,4.04,2.12,8.811,0.974,0.004973,9.241,344.004,7.124,0.050
2006-01-03,49.2427,4.03882,8.57,9.21,8.96,0.742,0.000,0.008280,7.535,382.930,8.292,0.160
2006-01-04,49.2427,4.03882,9.87,14.17,12.68,0.470,0.000,0.008134,19.244,356.590,5.446,0.428
2006-01-05,49.2427,4.03882,7.59,11.51,8.96,0.000,0.000,0.006754,14.233,367.906,2.020,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...
2100-12-27,49.2427,4.03882,0.67,4.61,2.63,0.000,0.000,0.003508,53.907,263.422,3.435,0.591
2100-12-28,49.2427,4.03882,-0.74,2.53,0.72,1.015,0.644,0.003386,12.878,314.804,3.790,0.432
2100-12-29,49.2427,4.03882,-1.10,1.20,-0.24,0.381,0.381,0.003521,18.212,310.789,1.192,0.185


In [18]:
data_rcp_4_5 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp4.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
Date,,,,,,,,,,,,
2006-01-01,49.2427,4.03882,-3.15,4.65,-0.02,0.220,0.220,3.503,53.157,264.926,7.70,0.156
2006-01-02,49.2427,4.03882,0.04,4.06,2.03,8.361,1.039,5.015,9.143,347.306,25.24,0.047
2006-01-03,49.2427,4.03882,8.59,8.97,8.82,0.677,0.000,8.196,8.756,381.686,30.14,0.171
2006-01-04,49.2427,4.03882,9.76,13.06,11.92,0.093,0.000,7.773,16.775,355.457,20.89,0.473
2006-01-05,49.2427,4.03882,5.15,11.00,7.20,0.000,0.000,6.088,29.116,337.960,5.61,0.191
...,...,...,...,...,...,...,...,...,...,...,...,...
2100-12-27,49.2427,4.03882,1.00,9.48,4.55,0.000,0.000,4.967,51.618,286.619,11.37,0.200
2100-12-28,49.2427,4.03882,4.14,11.98,8.26,3.633,0.000,5.910,38.051,318.954,14.50,0.370
2100-12-29,49.2427,4.03882,4.61,9.58,8.07,2.746,0.000,5.180,30.541,324.057,15.67,0.614


In [19]:
data_rcp_8_5 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp8.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
Date,,,,,,,,,,,,
2006-01-01,49.2427,4.03882,-3.15,4.58,-0.05,0.000,0.000,3.506,53.708,264.940,7.87,0.156
2006-01-02,49.2427,4.03882,0.05,4.04,2.02,8.608,1.052,4.984,9.463,344.383,25.13,0.047
2006-01-03,49.2427,4.03882,8.56,8.86,8.74,0.677,0.000,8.160,8.174,381.568,30.01,0.176
2006-01-04,49.2427,4.03882,9.76,12.29,11.42,0.111,0.000,7.996,16.040,357.124,21.37,0.248
2006-01-05,49.2427,4.03882,7.13,10.79,8.41,0.000,0.000,6.506,13.623,365.049,7.52,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...
2100-12-27,49.2427,4.03882,-0.66,6.08,1.58,0.070,0.000,3.103,51.761,285.622,7.58,0.496
2100-12-28,49.2427,4.03882,1.82,4.38,3.04,0.265,0.000,4.683,21.316,340.201,1.68,0.139
2100-12-29,49.2427,4.03882,2.17,6.72,5.15,0.463,0.136,4.927,17.369,342.966,8.96,0.158


##### **Dataset Prunay**
🌦️ Description des colonnes - Station météo Reims - Prunay

In [20]:
data_prunay = pd.read_csv('Atmo\donnees_Celine_LJ.csv')
data_prunay.head()

,time,latitude,longitude,10m-U_wind_ms,10m-V_wind_ms,2m_DewPoint_Temperature_K,2m_Temperature_K,Surface_Pressure_Pa,RH,Wind_Speed,Wind_Direction_CMEMS,Wind_Direction_Charbel,Wind_Direction_Era5
0,2006-01-01 00:00:00,49.24,4.04,3.134243,3.244023,275.30566,276.48438,98688.500,98.142507,4.510783,224.013944,225.986056,224.013944
1,2006-01-01 01:00:00,49.24,4.04,3.309480,3.341477,275.23462,276.61914,98683.940,97.822016,4.702991,224.724353,225.275647,224.724353
2,2006-01-01 02:00:00,49.24,4.04,3.143404,3.604999,275.22950,276.68042,98711.625,97.719006,4.782992,221.087024,228.912976,221.087024
3,2006-01-01 03:00:00,49.24,4.04,2.849324,3.933670,274.98290,276.33423,98712.940,97.871491,4.857202,215.917410,234.082590,215.917410
4,2006-01-01 04:00:00,49.24,4.04,2.640510,4.301672,275.22095,276.55884,98708.875,97.894295,5.047442,211.542989,238.457011,211.542989


In [21]:
data_prunay.dtypes

time                          object
latitude                     float64
longitude                    float64
10m-U_wind_ms                float64
10m-V_wind_ms                float64
2m_DewPoint_Temperature_K    float64
2m_Temperature_K             float64
Surface_Pressure_Pa          float64
RH                           float64
Wind_Speed                   float64
Wind_Direction_CMEMS         float64
Wind_Direction_Charbel       float64
Wind_Direction_Era5          float64
dtype: object

In [22]:
data_prunay['time'] = pd.to_datetime(data_prunay['time'], format='%Y-%m-%d %H:%M:%S')
data_prunay.set_index('time', inplace=True)
data_prunay.head()

,latitude,longitude,10m-U_wind_ms,10m-V_wind_ms,2m_DewPoint_Temperature_K,2m_Temperature_K,Surface_Pressure_Pa,RH,Wind_Speed,Wind_Direction_CMEMS,Wind_Direction_Charbel,Wind_Direction_Era5
time,,,,,,,,,,,,
2006-01-01 00:00:00,49.24,4.04,3.134243,3.244023,275.30566,276.48438,98688.500,98.142507,4.510783,224.013944,225.986056,224.013944
2006-01-01 01:00:00,49.24,4.04,3.309480,3.341477,275.23462,276.61914,98683.940,97.822016,4.702991,224.724353,225.275647,224.724353
2006-01-01 02:00:00,49.24,4.04,3.143404,3.604999,275.22950,276.68042,98711.625,97.719006,4.782992,221.087024,228.912976,221.087024
2006-01-01 03:00:00,49.24,4.04,2.849324,3.933670,274.98290,276.33423,98712.940,97.871491,4.857202,215.917410,234.082590,215.917410
2006-01-01 04:00:00,49.24,4.04,2.640510,4.301672,275.22095,276.55884,98708.875,97.894295,5.047442,211.542989,238.457011,211.542989


In [23]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [24]:
# récupérer les index communs data rcp4.5
common_index_data_rcp_4_5 = data_rcp_4_5.index.intersection(data_prunay.index)
data_rcp_4_5_ = data_rcp_4_5.loc[common_index_data_rcp_4_5]

In [25]:
# récupérer les index communs data rcp8.5
common_index_data_rcp_8_5 = data_rcp_8_5.index.intersection(data_prunay.index)
data_rcp_8_5_ = data_rcp_8_5.loc[common_index_data_rcp_8_5]

In [26]:
data_prunay.drop(['latitude', 'longitude'], axis=1, inplace=True)

In [27]:
data_prunay = data_prunay[["2m_Temperature_K", "RH"]].copy()

In [38]:
data_prunay.rename(columns={"RH": "prunay_RH"}, inplace=True)

In [28]:
data_prunay['2m_Temperature_C'] = data_prunay['2m_Temperature_K'] - 273.15
data_prunay.drop(['2m_Temperature_K'], axis=1, inplace=True)

In [39]:
data_prunay

,prunay_RH,2m_Temperature_C
time,,
2006-01-01 00:00:00,98.142507,3.33438
2006-01-01 01:00:00,97.822016,3.46914
2006-01-01 02:00:00,97.719006,3.53042
2006-01-01 03:00:00,97.871491,3.18423
2006-01-01 04:00:00,97.894295,3.40884
...,...,...
2025-06-12 04:00:00,93.596486,12.86025
2025-06-12 05:00:00,92.706792,14.57950
2025-06-12 06:00:00,90.352953,17.18080


In [40]:
data_rcp_2_6

,Latitude,Longitude,tasminAdjust,tasmaxAdjust,tasAdjust,prtotAdjust,prsnAdjust,hussAdjust,rsdsAdjust,rldsAdjust,sfcWindAdjust,evspsblpotAdjust
Date,,,,,,,,,,,,
2006-01-01,49.2427,4.03882,-3.15,4.59,-0.04,0.000,0.000,0.003508,53.716,265.013,2.181,0.155
2006-01-02,49.2427,4.03882,0.24,4.04,2.12,8.811,0.974,0.004973,9.241,344.004,7.124,0.050
2006-01-03,49.2427,4.03882,8.57,9.21,8.96,0.742,0.000,0.008280,7.535,382.930,8.292,0.160
2006-01-04,49.2427,4.03882,9.87,14.17,12.68,0.470,0.000,0.008134,19.244,356.590,5.446,0.428
2006-01-05,49.2427,4.03882,7.59,11.51,8.96,0.000,0.000,0.006754,14.233,367.906,2.020,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...
2100-12-27,49.2427,4.03882,0.67,4.61,2.63,0.000,0.000,0.003508,53.907,263.422,3.435,0.591
2100-12-28,49.2427,4.03882,-0.74,2.53,0.72,1.015,0.644,0.003386,12.878,314.804,3.790,0.432
2100-12-29,49.2427,4.03882,-1.10,1.20,-0.24,0.381,0.381,0.003521,18.212,310.789,1.192,0.185


In [ ]:
RH = convertir_q_en_rh(data_rcp_2_6["hussAdjust"], data_rcp_2_6["tasAdjust"])
data_rcp_2_6["rcp_RH"] = RH

In [37]:
data_rcp_2_6 = data_rcp_2_6[["RH", "tasAdjust"]].copy()

KeyError: "['RH'] not in index"